# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [8]:
!pip install selenium

In [14]:
# chrome -> chromedriver
# firefox -> geckodriver

from selenium import webdriver # controlar o navegador
from selenium.webdriver.common.keys import Keys # usar o teclado
from selenium.webdriver.common.by import By # localizar os itens no navegador 

# criar o navegador
navegador = webdriver.Chrome()

# entraria no google e pesquisaria por cotação do dólar
navegador.get("https://www.google.com/")

#pegaria a cotação do dólar  
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)


cotacao_dolar = navegador. find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


print(cotacao_dolar)

# entrarta no google e pesquisarta a cotação do euro
navegador.get("https://www.google.com/")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotaçao euro")

navegador. find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegarta a cotação do euro
cotacao_euro = navegador. find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#entrar no https://www.melhorcambio.com/ouro-hoje
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador. find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(",",".")

# pegarta a cotação do ouro
print(cotacao_ouro)

navegador.quit()

5.011299999999999
5.519120085
324.63


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [15]:
# importar a base de dodos

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

Produtos  Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar        5          4999.95    1.40   
1   Carro Renault         4500.00   Euro        6         27000.00    2.00   
2   Notebook Dell          899.99  Dólar        5          4499.95    1.70   
3          IPhone          799.00  Dólar        5          3995.00    1.70   
4      Carro Fiat         3000.00   Euro        6         18000.00    1.90   
5  Celular Xiaomi          480.48  Dólar        5          2402.40    2.00   
6        Joia 20g           20.00   Ouro      350          7000.00    1.15   

   Preço de Venda  
0        6999.930  
1       54000.000  
2        7649.915  
3        6791.500  
4       34200.000  
5        4804.800  
6        8050.000

- Atualizando os preços e o cálculo do Preço Final

In [20]:
# atual izar as cotações na base de dados
#cotacao do dolar
tabela.loc[tabela["Moeda"] == "Dólar" ,"Cotação"] = float(cotacao_dolar)
#cotacao do euro
tabela.loc[tabela["Moeda"] == "Euro" ,"Cotação"] = float(cotacao_euro)
#cotacao do ouro
tabela.loc[tabela["Moeda"] == "Ouro" ,"Cotação"] = float(cotacao_ouro)

# atualizar o preço de compra e de venda na base de dados
# preco de compra - preco original * cotacao
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
# preco de venda preco de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

Produtos  Preço Original  Moeda    Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar    5.01130      5011.249887    1.40   
1   Carro Renault         4500.00   Euro    5.51912     24836.040382    2.00   
2   Notebook Dell          899.99  Dólar    5.01130      4510.119887    1.70   
3          IPhone          799.00  Dólar    5.01130      4004.028700    1.70   
4      Carro Fiat         3000.00   Euro    5.51912     16557.360255    1.90   
5  Celular Xiaomi          480.48  Dólar    5.01130      2407.829424    2.00   
6        Joia 20g           20.00   Ouro  324.63000      6492.600000    1.15   

   Preço de Venda  
0     7015.749842  
1    49672.080765  
2     7667.203808  
3     6806.848790  
4    31458.984484  
5     4815.658848  
6     7466.490000

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
# exportar essa base de dados para ter o resultado atualizado
tabela.to_excel("ProdutoNovo.xlsx",index= False)